In [1]:
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.optim as optim
from PIL import Image
import torchvision.transforms as transforms
import wandb

/ocean/projects/cis250019p/mzhang23/TA/HW3P2/envs/hw3p2_env/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/ocean/projects/cis250019p/mzhang23/TA/HW3P2/envs/hw3p2_env/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` met

In [2]:
"""
EEG Multimodal Dataset for Classification Task
"""

class EEGMultimodalDataset(Dataset):
    """
    PyTorch Dataset for EEG-Image-Caption alignment.
    
    Returns: (eeg, image, caption, category_label, subject_id)
    - eeg: torch.FloatTensor of shape (122, 500)
    - image: torch.FloatTensor of shape (3, 224, 224)
    - caption: str
    - category_label: int (0-19)
    - subject_id: int (0-12)
    """
    
    def __init__(self, 
                 bids_root,
                 images_dir,
                 captions_path,
                 subject_list,
                 session_list,
                 image_transform=None,
                 clamp_thres=500,
                 normalization_stats=None  # Pass (mean, std) from training set
                ):
        
        self.bids_root = bids_root
        self.images_dir = images_dir
        self.image_transform = image_transform
        self.clamp_thres = clamp_thres
        
        # Create subject mapping (all 13 subjects)
        all_subjects = ['sub-02', 'sub-03', 'sub-05', 'sub-09', 'sub-14', 
                        'sub-15', 'sub-17', 'sub-19', 'sub-20', 'sub-23', 
                        'sub-24', 'sub-28', 'sub-29']
        self.subject_to_idx = {sub: idx for idx, sub in enumerate(all_subjects)}
        
        # Load captions and create category mapping
        print(f"Loading captions from {captions_path}...")
        self.captions_dict = self._load_captions(captions_path)
        
        # Extract unique categories and create mapping
        categories = sorted(set(cat for cat, _ in self.captions_dict.values()))
        self.category_to_idx = {cat: idx for idx, cat in enumerate(categories)}
        self.idx_to_category = {idx: cat for cat, idx in self.category_to_idx.items()}
        
        print(f"Loaded {len(self.captions_dict)} captions across {len(categories)} categories")
        print(f"Categories: {categories}")
        
        # Store metadata instead of raw data (memory efficient)
        self.trial_metadata = []
        
        print("Scanning dataset files...")
        for sub in subject_list:
            for ses in session_list:
                for run in ['01', '02', '03', '04']:
                    session_path = os.path.join(self.bids_root, sub, ses)
                    csv_path = os.path.join(session_path, 
                                           f"{sub}_{ses}_task-lowSpeed_run-{run}_image.csv")
                    npy_path = os.path.join(session_path, 
                                           f"{sub}_{ses}_task-lowSpeed_run-{run}_1000Hz.npy")
                    
                    if not (os.path.exists(csv_path) and os.path.exists(npy_path)):
                        print(f"Warning: Missing files for {sub} {ses} run-{run}")
                        continue
                    
                    try:
                        csv_data = pd.read_csv(csv_path)
                        # Just check shape, don't load all EEG data
                        eeg_shape = np.load(npy_path, mmap_mode='r').shape
                        
                        if eeg_shape[0] != len(csv_data):
                            print(f"Warning: Trial mismatch in {sub} {ses} run-{run}")
                            continue
                        
                        # Store metadata for each trial
                        for trial_idx, row in csv_data.iterrows():
                            img_base_name = self._get_base_name(row['FilePath'])
                            if not img_base_name:
                                continue
                            
                            category, caption = self.captions_dict.get(img_base_name, (None, None))
                            if category is None:
                                continue
                            
                            img_path = self._find_image_path(img_base_name)
                            if not img_path:
                                continue
                            
                            # Store complete metadata
                            self.trial_metadata.append({
                                'subject': sub,
                                'subject_id': self.subject_to_idx[sub],
                                'session': ses,
                                'run': run,
                                'trial_idx': trial_idx,
                                'npy_path': npy_path,
                                'img_path': img_path,
                                'category': category,
                                'category_label': self.category_to_idx[category],
                                'caption': caption
                            })
                    
                    except Exception as e:
                        print(f"Error processing {sub} {ses} run-{run}: {e}")
                        continue
        
        print(f"Found {len(self.trial_metadata)} valid trials")
        
        # Handle normalization statistics
        if normalization_stats is None:
            print("Computing normalization statistics for training set")
            self.norm_mean, self.norm_std = self._compute_normalization_stats()
        else:
            print("Using provided normalization statistics")
            self.norm_mean, self.norm_std = normalization_stats
    
    def _compute_normalization_stats(self):
        """
        Compute mean/std across all trials.
        """
        print("Loading sample of EEG data to compute statistics...")
        all_eeg = []
        
        # Sample every 10th trial to speed up (or use all for accuracy)
        sample_indices = range(0, len(self.trial_metadata), 10)
        
        for idx in sample_indices:
            meta = self.trial_metadata[idx]
            eeg_data = np.load(meta['npy_path'])[meta['trial_idx']]
            eeg_data = np.clip(eeg_data, -self.clamp_thres, self.clamp_thres)
            all_eeg.append(eeg_data)
        
        eeg_array = np.array(all_eeg, dtype=np.float32)  # (N, 122, 500)
        
        # Compute per-channel statistics
        mean = eeg_array.mean(axis=(0, 2))  # Shape: (122,)
        std = eeg_array.std(axis=(0, 2)) + 1e-6  # Shape: (122,)
        
        return mean, std
    
    def _load_captions(self, captions_path):
        """Load captions.txt into dictionary"""
        captions_dict = {}
        with open(captions_path, 'r') as f:
            next(f)  # Skip header
            for line in f:
                parts = line.strip().split('\t')
                if len(parts) >= 4:
                    category, img_name = parts[1], parts[2]
                    caption = '\t'.join(parts[3:]).strip()
                    captions_dict[img_name] = (category, caption)
        return captions_dict
    
    def _get_base_name(self, file_path):
        """Extract base image name, removing _resized suffix"""
        try:
            normalized_path = str(file_path).replace('\\', '/')
            base_name_with_ext = os.path.basename(normalized_path)
            base_name_resized = os.path.splitext(base_name_with_ext)[0]
            
            # Remove '_resized' suffix if present
            if base_name_resized.endswith('_resized'):
                base_name = base_name_resized[:-8]
            else:
                base_name = base_name_resized
            
            return base_name
        except Exception as e:
            return None
    
    def _find_image_path(self, img_base_name):
        """Find image file with any extension"""
        for ext in ['.jpg', '.jpeg', '.png', '.JPEG']:
            img_path = os.path.join(self.images_dir, img_base_name + ext)
            if os.path.exists(img_path):
                return img_path
        return None
    
    def __len__(self):
        return len(self.trial_metadata)
    
    def __getitem__(self, idx):
        """
        Returns: (eeg_tensor, image_tensor, caption, category_label, subject_id)
        
        Shapes:
        - eeg_tensor: (122, 500)
        - image_tensor: (3, 224, 224)
        - caption: string
        - category_label: int (0-19)
        - subject_id: int (0-12)
        """
        meta = self.trial_metadata[idx]
        
        # Load EEG on-demand (memory efficient)
        eeg_data = np.load(meta['npy_path'])[meta['trial_idx']]  # (122, 500)
        
        # Preprocess EEG
        eeg_data = np.clip(eeg_data, -self.clamp_thres, self.clamp_thres)
        
        # Normalize using per-channel statistics
        # Broadcasting: (122, 500) - (122, 1) / (122, 1)
        eeg_data = (eeg_data - self.norm_mean[:, None]) / self.norm_std[:, None]
        
        # Convert to tensor with correct shape (122, 500)
        eeg_tensor = torch.tensor(eeg_data, dtype=torch.float32)
        
        # Load image
        try:
            image = Image.open(meta['img_path']).convert('RGB')
            if self.image_transform:
                image_tensor = self.image_transform(image)
            else:
                image_tensor = transforms.ToTensor()(image)
        except Exception as e:
            print(f"Error loading {meta['img_path']}: {e}")
            image_tensor = torch.zeros(3, 224, 224)
        
        # Return all necessary data for classification
        return (
            eeg_tensor,                  # (122, 500)
            image_tensor,                # (3, 224, 224)
            meta['caption'],             # string
            meta['category_label'],      # integer 0-19
            meta['subject_id']           # integer 0-12
        )
    
    def get_normalization_stats(self):
        """Return normalization parameters for reuse in val/test sets"""
        return (self.norm_mean, self.norm_std)


def create_datasets(bids_root, images_dir, captions_path, all_subjects=None):
    """
    Factory function to create properly split datasets with no data leakage.
    
    Returns:
        train_dataset, val_dataset, test_dataset
    """
    if all_subjects is None:
        all_subjects = ['sub-02', 'sub-03', 'sub-05', 'sub-09', 'sub-14', 
                        'sub-15', 'sub-17', 'sub-19', 'sub-20', 'sub-23', 
                        'sub-24', 'sub-28', 'sub-29']
    
    # Session splits (from paper)
    train_sessions = ['ses-01', 'ses-02', 'ses-03']
    val_sessions = ['ses-04']
    test_sessions = ['ses-05']
    
    # Image transforms
    img_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])
    
    # Create training set FIRST (to compute normalization)
    print("\n" + "="*60)
    print("Creating TRAINING dataset...")
    print("="*60)
    train_dataset = EEGMultimodalDataset(
        bids_root=bids_root,
        images_dir=images_dir,
        captions_path=captions_path,
        subject_list=all_subjects,
        session_list=train_sessions,
        image_transform=img_transform
    )
    
    # Get normalization stats from training data
    norm_stats = train_dataset.get_normalization_stats()
    print(f"Training normalization: mean shape={norm_stats[0].shape}, "
          f"std shape={norm_stats[1].shape}")
    
    # Create val/test with SAME normalization (no data leakage)
    print("\n" + "="*60)
    print("Creating VALIDATION dataset...")
    print("="*60)
    val_dataset = EEGMultimodalDataset(
        bids_root=bids_root,
        images_dir=images_dir,
        captions_path=captions_path,
        subject_list=all_subjects,
        session_list=val_sessions,
        image_transform=img_transform,
        normalization_stats=norm_stats
    )
    
    print("\n" + "="*60)
    print("Creating TEST dataset...")
    print("="*60)
    test_dataset = EEGMultimodalDataset(
        bids_root=bids_root,
        images_dir=images_dir,
        captions_path=captions_path,
        subject_list=all_subjects,
        session_list=test_sessions,
        image_transform=img_transform,
        normalization_stats=norm_stats
    )
    
    # Print summary
    print("\n" + "="*60)
    print("DATASET SUMMARY")
    print("="*60)
    print(f"Train samples: {len(train_dataset):,}")
    print(f"Val samples:   {len(val_dataset):,}")
    print(f"Test samples:  {len(test_dataset):,}")
    print(f"Total:         {len(train_dataset) + len(val_dataset) + len(test_dataset):,}")
    print(f"\nCategories: {len(train_dataset.category_to_idx)}")
    print(f"Subjects:   {len(train_dataset.subject_to_idx)}")
    print("="*60)
    
    return train_dataset, val_dataset, test_dataset

In [3]:
train_ds, val_ds, test_ds = create_datasets(
    bids_root='/ocean/projects/cis250019p/gandotra/11785-gp-eeg/ds005589',
    images_dir='/ocean/projects/cis250019p/gandotra/11785-gp-eeg/images',
    captions_path='/ocean/projects/cis250019p/gandotra/11785-gp-eeg/captions.txt'
)

# Create dataloaders
train_loader = DataLoader(
    train_ds, 
    batch_size=32, 
    shuffle=True, 
    num_workers=4,
    pin_memory=True
)

val_loader = DataLoader(
    val_ds, 
    batch_size=32, 
    shuffle=False, 
    num_workers=4,
    pin_memory=True
)

test_loader = DataLoader(
    test_ds, 
    batch_size=32, 
    shuffle=False, 
    num_workers=4,
    pin_memory=True
)

print(f"DataLoaders created")
print(f"   Train batches: {len(train_loader)}")
print(f"   Val batches:   {len(val_loader)}")
print(f"   Test batches:  {len(test_loader)}")


Creating TRAINING dataset...
Loading captions from /ocean/projects/cis250019p/gandotra/11785-gp-eeg/captions.txt...
Loaded 9825 captions across 20 categories
Categories: ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'flower', 'horse', 'motorbike', 'person', 'sheep', 'sofa', 'train', 'tvmonitor']
Scanning dataset files...
Found 15600 valid trials
Computing normalization statistics for training set
Loading sample of EEG data to compute statistics...
Training normalization: mean shape=(500,), std shape=(500,)

Creating VALIDATION dataset...
Loading captions from /ocean/projects/cis250019p/gandotra/11785-gp-eeg/captions.txt...
Loaded 9825 captions across 20 categories
Categories: ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'flower', 'horse', 'motorbike', 'person', 'sheep', 'sofa', 'train', 'tvmonitor']
Scanning dataset files...
Found 5200 valid trials
Using pro

In [4]:
class ModelFC(nn.Module):
    def __init__(self, in_dim, hid_dim, out_dim,):
        super(ModelFC, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(in_dim, hid_dim),
            nn.BatchNorm1d(num_features=hid_dim),
            nn.ReLU(),
            nn.Linear(hid_dim, out_dim),
            nn.Softmax(dim=-1),
        )  
    def forward(self, x):
        # bs * time * electrode
        x_flat = x.reshape(x.shape[0], -1)
        return self.model(x_flat)

In [5]:
model = ModelFC(in_dim=122*500, hid_dim=512, out_dim=20)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())

In [ ]:
from tqdm import tqdm

def train_epoch(model, loader, criterion, optimizer):
    """Train for one epoch"""
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    pbar = tqdm(loader, desc='Training')
    for batch_idx, batch in enumerate(pbar):
        # Unpack batch - adjust based on what your dataloader returns
        if len(batch) == 5:
            eeg, _, _, labels, subject_ids = batch
        else:
            eeg, _, labels = batch  # If using old dataloader
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(eeg)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Statistics
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        # Update progress bar
        if (batch_idx + 1) % 50 == 0:
            pbar.set_postfix({
                'loss': f'{total_loss/(batch_idx+1):.4f}',
                'acc': f'{100.*correct/total:.2f}%'
            })
    
    return total_loss / len(loader), 100. * correct / total


def validate(model, loader, criterion):
    """Validate the model"""
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in tqdm(loader, desc='Validating'):
            # Unpack batch
            if len(batch) == 5:
                eeg, _, _, labels, _ = batch
            else:
                eeg, _, labels = batch
            
            # Forward pass
            outputs = model(eeg)
            loss = criterion(outputs, labels)
            
            # Statistics
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    return total_loss / len(loader), 100. * correct / total

In [ ]:
import wandb

wandb.login(key="825201e63a02e53435b53a136158ab39815c89a4")
# Training configuration
config = {
    'batch_size': 32,
    'learning_rate': 1e-4,
    'weight_decay': 1e-5,
    'num_epochs': 100,
    'model_type': 'fc_baseline',
    'optimizer': 'Adam',
    'loss_fn': 'CrossEntropy'
}

# Initialize wandb
wandb.init(
    project="eeg-classification",
    name="baseline-run-3-100epochs",  # Change this for different runs
    config=config,
    tags=['baseline', 'midterm']
)

# Create model
model = ModelFC(in_dim=122*500, hid_dim=512, out_dim=20)

# Move to GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)
print(f"✅ Model on {device}")

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(
    model.parameters(), 
    lr=config['learning_rate'], 
    weight_decay=config['weight_decay']
)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
wandb.config.update({'total_params': total_params})
print(f"   Total parameters: {total_params:,}")

# Watch model with wandb (logs gradients and parameters)
wandb.watch(model, criterion, log="all", log_freq=100)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


batch_train_acc,▁▁▁▁▁▂▂▃▃▃▃▄▄▄▅▅▆▆▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇████
batch_train_loss,██████▇█▇▇▅▇▇▄▆▅▆▅▃▄▅▄▄▄▅▃▃▂▁▃▁▂▂▃▃▄▃▂▃▃
class_0_acc,▁▂▃▄▃▃▃▅▄▄▆▄▄▃▄▄▅▄▄▃▅▆▆▄▄▆▄▇█▄▅▅▅▆▅▆▆▅▃▅
class_10_acc,▁▁▃▁▆▃▄▆▅▇▄▃▄▄▄▇▃▅▄▆▄▅█▅▅▅▃▄▅▆▅▃▆▇▆▆█▇▆▅
class_11_acc,▁▂▃▂▇▁▄▁▂▁▄▄▃▂▂▂▂▄▁▄▅▄▂▁█▂▅▂▁▃▂▅▁▅▃▅▅▃▄▂
class_12_acc,▅▄▄▇▆▄▆▆▇▄█▇▆▅▅█▅▄▄▅▄▄▃▅▂▄▃▄▆▂▅▁▃▃▂▂▃▂▂▂
class_13_acc,▁▁▁▁▁▅▃▃▃▄▂▄▃▅▃█▅▅▄▃▆▅▃▄▄▃▃▃▄▅▃▄▃▃▃▄▅▄▅▆
class_14_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▆█▆
class_15_acc,▃▅█▅▅█▇▅█▅▃▄▄▄▃▄▃▃▆▂▂▂▄▅▃▂▂▂▂▃▂▂▂▂▂▁▂▂▂▂
class_16_acc,▂▂▁▄▃▂▅▃▆▆▁▃█▇▄▄▄▆▄▃▅▅▅▃▆▄▄▃▅█▃▄▃▇▇▄▆▄█▄
+18,...


✅ Model on cuda
   Total parameters: 31,243,796


In [15]:
from tqdm import tqdm

def train_epoch(model, loader, criterion, optimizer, device, epoch):
    """Train for one epoch with wandb logging"""
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    pbar = tqdm(loader, desc=f'Epoch {epoch}')
    for batch_idx, batch in enumerate(pbar):
        # Unpack batch
        if len(batch) == 5:
            eeg, _, _, labels, subject_ids = batch
        else:
            eeg, _, labels = batch
        
        eeg = eeg.to(device)
        labels = labels.to(device)
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(eeg)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Statistics
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        # Log to wandb every 10 batches
        if batch_idx % 10 == 0:
            wandb.log({
                'batch_train_loss': loss.item(),
                'batch_train_acc': 100. * correct / total,
                'learning_rate': optimizer.param_groups[0]['lr']
            })
        
        # Update progress bar
        if (batch_idx + 1) % 50 == 0:
            pbar.set_postfix({
                'loss': f'{total_loss/(batch_idx+1):.4f}',
                'acc': f'{100.*correct/total:.2f}%'
            })
    
    epoch_loss = total_loss / len(loader)
    epoch_acc = 100. * correct / total
    
    return epoch_loss, epoch_acc


def validate(model, loader, criterion, device):
    """Validate the model"""
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    # Per-class statistics
    class_correct = [0] * 20
    class_total = [0] * 20
    
    with torch.no_grad():
        for batch in tqdm(loader, desc='Validating'):
            # Unpack batch
            if len(batch) == 5:
                eeg, _, _, labels, _ = batch
            else:
                eeg, _, labels = batch
            
            eeg = eeg.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(eeg)
            loss = criterion(outputs, labels)
            
            # Statistics
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            # Per-class accuracy
            for i in range(len(labels)):
                label = labels[i].item()
                class_correct[label] += (predicted[i] == labels[i]).item()
                class_total[label] += 1
    
    epoch_loss = total_loss / len(loader)
    epoch_acc = 100. * correct / total
    
    # Calculate per-class accuracy
    class_acc = {}
    for i in range(20):
        if class_total[i] > 0:
            class_acc[f'class_{i}_acc'] = 100. * class_correct[i] / class_total[i]
    
    return epoch_loss, epoch_acc, class_acc

print("✅ Training functions defined")

✅ Training functions defined


In [16]:
import os

# Create checkpoint directory
os.makedirs('checkpoints', exist_ok=True)

# Training variables
num_epochs = config['num_epochs']
best_val_acc = 0

print("🚀 Starting training...\n")

for epoch in range(1, num_epochs + 1):
    print(f"\nEpoch {epoch}/{num_epochs}")
    print("-" * 50)
    
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device, epoch)
    
    # Validate
    val_loss, val_acc, class_acc = validate(model, val_loader, criterion, device)
    
    # Log epoch metrics to wandb
    wandb.log({
        'epoch': epoch,
        'train_loss': train_loss,
        'train_acc': train_acc,
        'val_loss': val_loss,
        'val_acc': val_acc,
        **class_acc  # Log per-class accuracy
    })
    
    # Print results
    print(f"\nResults:")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"  Val Loss:   {val_loss:.4f}   | Val Acc:   {val_acc:.2f}%")
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        
        # Save checkpoint
        checkpoint_path = 'checkpoints/best_model.pth'
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_acc': val_acc,
            'train_acc': train_acc,
        }, checkpoint_path)
        
        # Log model to wandb
        wandb.save(checkpoint_path)
        
        print(f"  ✅ New best model! Val Acc: {val_acc:.2f}%")
        
        # Log best metrics
        wandb.run.summary["best_val_acc"] = val_acc
        wandb.run.summary["best_epoch"] = epoch


print("\n🎉 Training complete!")
print(f"Best Val Acc: {best_val_acc:.2f}%")

🚀 Starting training...


Epoch 1/100
--------------------------------------------------


Validating: 100%|████████████████████████████████████████████████████████████████████████████| 163/163 [00:18<00:00,  8.96it/s]



Results:
  Train Loss: 2.9941 | Train Acc: 6.53%
  Val Loss:   2.9926   | Val Acc:   6.54%
  ✅ New best model! Val Acc: 6.54%

Epoch 2/100
--------------------------------------------------


Validating: 100%|████████████████████████████████████████████████████████████████████████████| 163/163 [00:18<00:00,  8.81it/s]



Results:
  Train Loss: 2.9807 | Train Acc: 9.17%
  Val Loss:   2.9893   | Val Acc:   7.81%
  ✅ New best model! Val Acc: 7.81%

Epoch 3/100
--------------------------------------------------


...


Results:
  Train Loss: 2.4296 | Train Acc: 66.58%
  Val Loss:   2.9860   | Val Acc:   8.29%

Epoch 99/100
--------------------------------------------------


Validating: 100%|████████████████████████████████████████████████████████████████████████████| 163/163 [00:18<00:00,  8.66it/s]



Results:
  Train Loss: 2.4309 | Train Acc: 66.56%
  Val Loss:   2.9852   | Val Acc:   8.15%

Epoch 100/100
--------------------------------------------------


Validating: 100%|████████████████████████████████████████████████████████████████████████████| 163/163 [00:18<00:00,  8.61it/s]


Results:
  Train Loss: 2.4241 | Train Acc: 67.12%
  Val Loss:   2.9880   | Val Acc:   7.88%

🎉 Training complete!
Best Val Acc: 8.98%
